In [ ]:
import os
import json
import datetime
import requests
import time
import pandas as pd

# パラメータの設定
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # 名前は適宜変更
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ 
    {"name": "openai1", "location": "eastus", "priority": 1, "weight": 80}, 
    {"name": "openai2", "location": "swedencentral", "priority": 1, "weight": 10}
] # OpenAIリソースのリスト
openai_resources_sku = "S0"
openai_model_name = "gpt-4o"
openai_model_version = "2024-08-06"
openai_deployment_name = "openAI-test"
openai_api_version = "2024-10-21"
openai_specification_url = f'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/{openai_api_version}/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ 
    {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, 
    {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} 
]

log_analytics_name = "workspace"
app_insights_name = 'insights'

app_registration_name = "ai-gateway-openai-app3"


<a id='1'></a>
### 1️⃣ Microsoft Entra IDでアプリ登録を作成
次のコマンドは、クライアントアプリケーションの登録を作成します。

In [ ]:
cmd_stdout = ! az account show --query homeTenantId --output tsv
tenant_id = cmd_stdout.n

cmd_stdout = ! az ad app create --display-name {app_registration_name} --query appId --is-fallback-public-client true --output tsv
client_id = cmd_stdout.n


<a id='2'></a>
### 2️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定したリソースグループ内に作成されます。既存のリソースグループを使用したい場合は、この手順をスキップしてください。

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if any("ERROR" in line for line in resource_group_stdout):
    print(resource_group_stdout)
else:
    print(f"✅ Azure Resource Group '{resource_group_name}' created at {datetime.datetime.now().time()}")


<a id='3'></a>
### 2️⃣ 🦾 Bicepを使ったデプロイメントの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。パラメータを変更するか、[main.bicep](main.bicep)を直接編集して、異なる構成を試すことができます。

`openAIModelCapacity`は意図的に低く設定されており、`2`（1分あたり2kトークン）に設定されています。これは、ロードバランサーのリトライロジックを示すためです。

In [ ]:
# バックエンドIDの設定
if len(openai_resources) > 1:
    backend_id = openai_backend_pool
elif len(openai_resources) == 1:
    backend_id = openai_resources[0].get("name")
elif len(mock_webapps) > 1:
    backend_id = mock_backend_pool
elif len(mock_webapps) == 1:
    backend_id = mock_webapps[0].get("name")
else:
    raise ValueError("No backend defined.")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id).replace("{aad-client-application-id}", client_id).replace("{aad-tenant-id}", tenant_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

# Bicepパラメータの作成
bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name },
    "apimResourceLocation": { "value": apim_resource_location },
    "apimSku": { "value": apim_resource_sku },
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}

# パラメータファイルの保存
with open('params.json', 'w') as bicep_parameters_file:
    json.dump(bicep_parameters, bicep_parameters_file, indent=2)

# Bicepテンプレートのデプロイ
deployment_stdout = ! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"
if any("ERROR" in line for line in deployment_stdout):
    print(deployment_stdout)
else:
    print(f"✅ Bicep Deployment '{deployment_name}' completed at {datetime.datetime.now().time()}")

# 元のpolicy.xmlに戻す（必要に応じて）
with open("policy.xml", 'w') as policy_xml_file:
    policy_xml_file.write(policy_template_xml)


<a id='4'></a>
### 4️⃣ デプロイメントの出力を取得

テスト準備が整う前に、ゲートウェイのURLとサブスクリプションを取得する段階に来ました。

In [ ]:
# APIM Subscription Keyの取得
subscription_key_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = subscription_key_stdout[0].strip()

# APIM Gateway URLの取得
gateway_url_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = gateway_url_stdout[0].strip()
print(f"👉🏻 API Gateway URL: {apim_resource_gateway_url}")

# Log Analytics Workspace IDの取得
workspace_id_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = workspace_id_stdout[0].strip()
print(f"👉🏻 Log Analytics Workspace ID: {workspace_id}")

# Application Insights App IDの取得
app_id_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = app_id_stdout[0].strip()
print(f"👉🏻 Application Insights App ID: {app_id}")


<a id='5'></a>
### 5️⃣ デバイスフローを作成してアクセストークンを取得

詳細な認可に関するメモ：
- APIMの[JWT検証ポリシー](https://learn.microsoft.com/en-us/azure/api-management/validate-azure-ad-token-policy)を使用して、特定のクレーム（トークンに存在する必要があるもの）をチェックし、詳細な認可を適用できます。
- グループクレームは一般的な方法です。このアプローチを使用して認可を行うことができます。ただし、ユーザーが多くのグループに属している場合、`groups`はトークンのサイズ制限によりトークンから除外されることがあります。
- 代替手段として、アプリケーションロール定義を構成し、ユーザーやグループをアプリロールに割り当てることができます。このZero Trust開発者のベストプラクティスは、柔軟性とコントロールを向上させ、最小権限でアプリケーションのセキュリティを強化します。[詳細はこちら](https://learn.microsoft.com/en-us/security/zero-trust/develop/configure-tokens-group-claims-app-roles)。
- `roles`クレームを取得するには、アプリ登録の「APIを公開」セクションに移動し、アプリケーションID URIとスコープを追加します。その後、完全なスコープ（app://<id>/scope）をコピーし、以下のスコープ配列に追加します。
- 「アプリロール」ブレードに移動し、ユーザー/グループメンバー用にアプリロール（例：OpenAI.ChatCompletion）を作成します。その後、テストユーザーまたはグループをアプリロールに割り当てます。
- ログイン後、https://jwt.io/ を使用して`access_token`変数をデコードし、`roles`が送信されているかどうかを確認します。
- 上記の構成で、APIMポリシーに以下のフラグメントを追加して、ユーザーが特定のアプリロールに属していることを確認できます：
```
            <required-claims>
                <claim name="roles" match="any">
                    <value>OpenAI.ChatCompletion</value>
                </claim>
            </required-claims>
```

In [ ]:
import json
import logging

import requests
import msal

app = msal.PublicClientApplication(
    client_id, authority="https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes=["User.Read"])
if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent=4))

print(flow["message"])

<a id='6'></a>
### 6️⃣ トークンを取得してGraph APIをクエリ



In [ ]:
result = app.acquire_token_by_device_flow(flow)
if "access_token" in result:
    access_token = result['access_token']
    # Calling graph using the access token
    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent=2))
    # print(access_token) # Use a tool like https://jwt.io/ to decode the access token and see its contents
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  #

<a id='requests'></a>
### 🧪 直接HTTP呼び出しを使用してAPIをテスト
`Requests`は、Python用の優れたシンプルなHTTPライブラリで、ここでは生のAPIリクエストを行い、レスポンスを確認するために使用されます。

In [ ]:
# リクエスト回数と間隔の設定
runs = 5
sleep_time_sec = 2

# APIエンドポイントの構築
url = f"{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"

for i in range(runs):
    print(f"▶️ Run: {i+1}")
    if len(openai_resources) > 0:
        messages = {
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Can you tell me the time, please?"}
            ]
        }
    elif len(mock_webapps) > 0:
        messages = {
            "messages": [
                {
                    "role": "system", 
                    "content": {
                        "simulation": {
                            "default": {"response_status_code": 200, "wait_time_ms": 0},
                            "openaimock1.azurewebsites.net": {"response_status_code": 429}
                        }
                    }
                }
            ]
        }
    else:
        messages = {}
    
    response = requests.post(url, headers = {'api-key':apim_subscription_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
    print(f"Status Code: {response.status_code}")
    print(f"Headers: {response.headers}")
    print(f"x-ms-region: {response.headers.get('x-ms-region')}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"Response: {data.get('choices')[0].get('message').get('content')}")
    else:
        print(f"Error Response: {response.text}")
    
    time.sleep(sleep_time_sec)


<a id='kql'></a>
### 🔍 Application Insightsリクエストの分析

このクエリを使用すると、リクエストおよびレスポンスの詳細（プロンプトとOpenAIの補完を含む）を取得できます。また、トークンカウンタも返されます。

In [ ]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
